In [1]:
### НУЖНЫЕ ИМПОРТЫ
!pip install python-dotenv==1.0.1 mistralai langchain_huggingface langchain_community chromadb

In [11]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
import re
import os
from mistralai import Mistral

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import json

#collab like
# from google.colab import userdata

## 1. USER QUERY

In [3]:
# Пока вопрос в виде строки, потом видимо хотим сделать как .py скрипт
user_query = "Как заселить НИП?"

## 2. RETRIEVAL
#### 2.1 BUILDING DOCS EMBEDDINGS

In [4]:
with open('wiki_dump_test_without_temps.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

inputs = [item["content"] for item in data if "content" in item]

In [5]:
documents = [Document(page_content=text) for text in inputs]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
chunks = text_splitter.split_documents(documents)

In [6]:
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cuda"}
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
vectorstore = Chroma.from_documents(chunks, embeddings, persist_directory="./terraria_db")

#### 2.2 MAKE RETRIEVER

In [8]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

#### 2.3 SEARCH

In [9]:
results = vectorstore.similarity_search_with_score(user_query, k=5)
docs = []

for (doc, score) in results:
    docs.append(doc.page_content)

## 3. PROMPT BUILDING



In [12]:
from google.colab import userdata

# Access the secret
api_key = userdata.get('MISTRAL_API_KEY')

if not api_key:
    raise ValueError("❌ MISTRAL_API_KEY not found in Colab secrets!")
else:
    print("✅ API key loaded successfully from Colab secrets!")

# Initialize Mistral client
client = Mistral(api_key=api_key)

# Test connection
def test_connection():
    try:
        models = client.models.list()
        print("✅ Connected successfully!")
        print(f"Available models: {[m.id for m in models.data]}")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        print("💡 If key is not active yet, wait a few minutes and try again")

test_connection()

✅ API key loaded successfully from Colab secrets!
✅ Connected successfully!
Available models: ['mistral-medium-2505', 'mistral-large-latest', 'mistral-medium-2508', 'mistral-medium-latest', 'mistral-medium', 'ministral-3b-2410', 'ministral-3b-latest', 'ministral-8b-2410', 'ministral-8b-latest', 'open-mistral-7b', 'mistral-tiny', 'mistral-tiny-2312', 'open-mistral-nemo', 'open-mistral-nemo-2407', 'mistral-tiny-2407', 'mistral-tiny-latest', 'mistral-large-2411', 'pixtral-large-2411', 'pixtral-large-latest', 'mistral-large-pixtral-2411', 'codestral-2508', 'codestral-latest', 'devstral-small-2507', 'devstral-small-latest', 'devstral-medium-2507', 'devstral-medium-latest', 'mistral-vibe-cli-latest', 'pixtral-12b-2409', 'pixtral-12b', 'pixtral-12b-latest', 'mistral-small-2506', 'mistral-small-latest', 'magistral-medium-2509', 'magistral-medium-latest', 'magistral-small-2509', 'magistral-small-latest', 'voxtral-mini-2507', 'voxtral-mini-latest', 'voxtral-small-2507', 'voxtral-small-latest', '

In [13]:
def run_mistral(messages, user_format=True, model="mistral-medium-latest"):
    client = Mistral(api_key=api_key)
    if user_format:
        messages = [
            {"role":"user", "content":messages}
        ]
    chat_response = client.chat.complete(
        model=model,
        messages=messages
    )
    return chat_response.choices[0].message.content

In [18]:
def user_messageRAG(query, docs):
    user_message = (
        f"""
        Ты эксперт по игре Terraria, ты должен ответить на вопрос пользователя

        ####
        Вот немного вспомогательной информации:

        <<<
        INFO: {docs[0]}
        >>>
        <<<
        INFO: {docs[1]}
        >>>
        <<<
        INFO: {docs[2]}
        >>>
        <<<
        INFO: {docs[3]}
        >>>
        <<<
        INFO: {docs[4]}
        >>>
        ###

        <<<
        Query: {query}
        >>>
        """
    )
    return user_message

## 4. LLM ANSWER

In [21]:
user_query = "Как скрафтить зелье подводного дыхания?"

In [22]:
model_answer = run_mistral(user_messageRAG(user_query, docs))


In [23]:
print(model_answer)

В *Terraria* **зелье подводного дыхания** (*Gills Potion*) позволяет дышать под водой в течение **2 минут** (или **4 минут** с аксессуаром *Алхимический цветок* или *Фляжка для зелий*).
Для его создания потребуются:

### **Рецепт крафта** (на верстаке или алхимическом столе):
- **2 ×** *Морской сорняк* (*Seaweed*) – собирается под водой (в океане, джунглях или любом водоёме).
- **1 ×** *Жабры* (*Gills*) – дроп с **Короля слизней** (*King Slime*, 25% шанс) или **Пираньи** (1% шанс в джунглях).
- **1 ×** *Бутылка* (*Bottle*) – крафтится из **стекла** (2 песка → 1 стекло → 1 бутылка).
- **1 ×** *Светящийся гриб* (*Glowing Mushroom*) – растёт в **грибном биоме** (или покупается у *Трактирщика* ночью за 20 медных монет).

---
### **Альтернативные способы получения:**
1. **Покупка у Заклинателя** (*Witch Doctor*) в **джунглях** (ночью, если связан с *Дриадой* или *Пиратом*).
   *Стоимость:* 2 золотых 50 серебряных монет (в жёстком режиме – дешевле).
2. **Находка в сундуках** (редко в подземн

In [24]:
def user_message(query):
    user_message = (
        f"""
        Ты эксперт по игре Terraria, ты должен ответить на вопрос пользователя

        <<<
        Query: {query}
        >>>
        """
    )
    return user_message

In [25]:
model_answer = run_mistral(user_message(user_query))
print(model_answer)

В **Terraria** **Зелье подводного дыхания (Gills Potion)** позволяет дышать под водой в течение **2 минут** (или дольше с аксессуарами, увеличивающими продолжительность зелий). Вот как его скрафтить:

---

### **Ингредиенты для крафта (1 зелье):**
| Предмет                     | Количество | Где добыть                                                                 |
|------------------------------|------------|---------------------------------------------------------------------------|
| **Жабры (Gill)**             | 1          | Выпадает с **Короля Слизней (King Slime)** (шанс 33.33% в обычном режиме, 100% в режиме Эксперт/Мастер). |
| **Ботва (Blinkroot)**        | 1          | Растёт в **Подземных джунглях (Underground Jungle)** или покупается у **Ведьмы (Witch Doctor)** ночью (50 медных монет). |
| **Бутылка (Bottle)**         | 1          | Крафтится из **стекла (Glass)** на верстаке или покупается у **Пьяницы (Demolitionist)**. |
| **Вода (Water)**             | 1          | На